In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2 # Using MobileNetV2 for efficiency
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import pathlib
import time




In [ ]:

# --- 1. Configuration and Constants ---
DATA_DIR = './PlantVillage/PlantVillage' # <--- MAKE SURE THIS PATH IS CORRECT
IMG_HEIGHT = 128          # Reduced size for faster training, can increase (e.g., 224)
IMG_WIDTH = 128
BATCH_SIZE = 32
EPOCHS = 25               # Number of epochs for initial training
FINE_TUNE_EPOCHS = 10     # Number of epochs for fine-tuning
LEARNING_RATE = 0.001
FINE_TUNE_LEARNING_RATE = 1e-5 # Very low learning rate for fine-tuning
VALIDATION_SPLIT = 0.2    # 20% of data for validation
SEED = 123                # For reproducibility

# Check if data directory exists
data_dir_path = pathlib.Path(DATA_DIR)
if not data_dir_path.exists():
    print(f"Error: Data directory '{DATA_DIR}' not found.")
    print("Please ensure the 'PlantVillage' folder is in the correct location.")
    exit() # Stop execution if data is missing

In [2]:

# --- 2. Load and Prepare Data ---
print(f"\nLoading data from: {DATA_DIR}")
print(f"Image size: {IMG_HEIGHT}x{IMG_WIDTH}")
print(f"Batch size: {BATCH_SIZE}")

# Use image_dataset_from_directory for efficient loading
train_ds = image_dataset_from_directory(
    data_dir_path,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='int' # Use integer labels for sparse_categorical_crossentropy
)

val_ds = image_dataset_from_directory(
    data_dir_path,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='int'
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"\nFound {num_classes} classes:")
# Print first 10 classes for brevity if too many
if num_classes > 10:
    print(class_names[:10], "...")
else:
    print(class_names)



Loading data from: ./PlantVillage/PlantVillage
Image size: 128x128
Batch size: 32
Found 20638 files belonging to 15 classes.
Using 16511 files for training.
Found 20638 files belonging to 15 classes.
Using 4127 files for validation.

Found 15 classes:
['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot'] ...


In [ ]:

# --- 3. Data Augmentation and Preprocessing Layers ---
# Create data augmentation layers
data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal", input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        # layers.RandomContrast(0.1) # Optional: Add more augmentation
    ],
    name="data_augmentation",
)

# Preprocessing layer (part of MobileNetV2 preprocessing)
# MobileNetV2 expects inputs in the range [-1, 1]
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Configure datasets for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# --- 4. Build Model using Transfer Learning (MobileNetV2) ---
print("\nBuilding model using MobileNetV2...")

# Load the base model (pre-trained on ImageNet)
# include_top=False removes the final classification layer
base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                         include_top=False,
                         weights='imagenet')

# Freeze the base model layers initially
base_model.trainable = False

# Create the full model
inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = data_augmentation(inputs)       # Apply augmentation
x = preprocess_input(x)             # Apply MobileNetV2 preprocessing
x = base_model(x, training=False)   # Run base model (batch norm layers in inference mode)
x = layers.GlobalAveragePooling2D()(x) # Pool features
x = layers.Dropout(0.3)(x)          # Regularization dropout
outputs = layers.Dense(num_classes, activation='softmax')(x) # Output layer

model = keras.Model(inputs, outputs)

# --- 5. Compile the Model ---
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='sparse_categorical_crossentropy', # Use sparse for integer labels
              metrics=['accuracy'])

print("\nInitial Model Summary (Base Model Frozen):")
model.summary()

# --- 6. Define Callbacks ---
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)
callbacks = [early_stopping, reduce_lr]

# --- 7. Initial Training (Feature Extraction) ---
print(f"\n--- Starting Initial Training (Epochs: {EPOCHS}) ---")
start_time = time.time()

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks
)

initial_training_time = time.time() - start_time
print(f"--- Initial Training Finished in {initial_training_time:.2f} seconds ---")

# --- 8. Fine-Tuning (Optional but Recommended) ---
print("\n--- Starting Fine-Tuning ---")

# Unfreeze the base model
base_model.trainable = True

# How many layers to unfreeze? Fine-tune from this layer onwards.
# A common practice is to unfreeze the top blocks. For MobileNetV2, ~ last 50 layers.
fine_tune_at = 100 # Unfreeze layers from index 100 onwards

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
print(f"Unfreezing layers from index {fine_tune_at} onwards in the base model.")

# Re-compile the model with a very low learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("\nModel Summary (During Fine-Tuning):")
model.summary() # Note the increased number of trainable parameters

# Adjust callbacks for fine-tuning if needed (e.g., lower patience)
fine_tune_callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True), # Keep similar patience or slightly lower
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1) # More sensitive LR reduction
]

# Continue training (fine-tuning)
total_epochs_run_initial = len(history.epoch)
print(f"\n--- Continuing Training for Fine-Tuning (Max Epochs: {FINE_TUNE_EPOCHS}) ---")
start_time_ft = time.time()

history_fine = model.fit(
    train_ds,
    epochs=total_epochs_run_initial + FINE_TUNE_EPOCHS, # Total epochs to run up to
    initial_epoch=total_epochs_run_initial, # Start counting from where initial training stopped
    validation_data=val_ds,
    callbacks=fine_tune_callbacks
)

fine_tuning_time = time.time() - start_time_ft
print(f"--- Fine-Tuning Finished in {fine_tuning_time:.2f} seconds ---")
print(f"--- Total Training Time: {initial_training_time + fine_tuning_time:.2f} seconds ---")


# --- 9. Evaluate the Model ---
print("\n--- Evaluating Final Model ---")

# Evaluate on validation set
final_loss, final_accuracy = model.evaluate(val_ds)
print(f"\nFinal Validation Loss: {final_loss:.4f}")
print(f"Final Validation Accuracy: {final_accuracy:.4f}")

# --- 10. Visualize Results ---

# Combine histories for plotting
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
# Mark where fine-tuning started
plt.axvline(total_epochs_run_initial -1 , linestyle='--', color='r', label='Start Fine-Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
# Mark where fine-tuning started
plt.axvline(total_epochs_run_initial -1, linestyle='--', color='r', label='Start Fine-Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.suptitle('Model Training History (Including Fine-Tuning)')
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
plt.show()


# --- Confusion Matrix and Classification Report ---
print("\nGenerating Confusion Matrix and Classification Report...")

# Get predictions for the entire validation set
y_pred_probabilities = model.predict(val_ds)
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Get true labels
y_true = []
# Iterate over the validation dataset batches
for images, labels in val_ds:
    y_true.extend(labels.numpy())

# Ensure we have the same number of predictions and true labels
if len(y_pred) != len(y_true):
     print(f"Warning: Mismatch in prediction ({len(y_pred)}) and true label ({len(y_true)}) counts. Re-extracting true labels.")
     # Re-extract true labels carefully, ensuring order matches prediction if possible
     # This might happen if the dataset size isn't perfectly divisible by batch size and predict doesn't drop remainder
     y_true = np.concatenate([y for x, y in val_ds], axis=0)
     # If still mismatch, there might be an issue with predict() or dataset iteration
     if len(y_pred) != len(y_true):
         print("Error: Cannot align predictions and labels. Skipping Confusion Matrix/Report.")
     else:
        print("Label count corrected.")


if len(y_pred) == len(y_true):
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(max(12, num_classes // 2), max(10, num_classes // 2.5))) # Adjust size dynamically
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.title('Confusion Matrix', fontsize=14)
    plt.xticks(rotation=90, fontsize=8) # Rotate labels for readability
    plt.yticks(rotation=0, fontsize=8)
    plt.tight_layout()
    plt.show()

    # Classification Report
    print("\nClassification Report:")
    # Use zero_division=0 to handle cases where a class might have no predicted samples
    print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))
else:
    print("Skipping Confusion Matrix and Classification Report due to label/prediction count mismatch.")


# --- 11. Save the Final Model (Optional) ---
# model.save('plant_village_mobilenetv2_final.h5')
# print("\nFinal model saved as 'plant_village_mobilenetv2_final.h5'")

# Or save in SavedModel format (recommended)
model.save('plant_village_mobilenetv2_savedmodel')
print("\nFinal model saved in SavedModel format as 'plant_village_mobilenetv2_savedmodel'")


print("\n--- Script Execution Complete ---")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Building model using MobileNetV2...

Initial Model Summary (Base Model Frozen):


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │        19,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,277,199 (8.69 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │        19,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,277,199 (8.69 MB)

 Trainable params: 1,880,655 (7.17 MB)

 Non-trainable params: 396,544 (1.51 MB)


--- Continuing Training for Fine-Tuning (Max Epochs: 10) ---
Epoch 26/35
451/516 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - accuracy: 0.5664 - loss: 2.9109